In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [ ]:
tf.config.set_visible_devices([], 'GPU')
print("Dispositivos tras deshabilitar GPUs:", tf.config.get_visible_devices())

In [ ]:
df = pd.read_csv('SolAtasIMC_tratado.csv')
df.head()

In [ ]:
print(df.info())

# Preprocesado de Datos

In [ ]:
tamanio = df.shape[0]

In [ ]:
df_train = df.copy().loc[0:int(tamanio*0.7)]
df_train

In [ ]:
df_vali = df.copy().loc[int(tamanio*0.7 + 1):int(tamanio*0.9)]
df_vali

In [ ]:
df_test = df.copy().loc[int(tamanio*0.9 + 1):tamanio]
df_test

In [ ]:
df_valitest = pd.concat([df_vali, df_test], axis=0)

Numero de horas que se utilizan en la predicción

In [ ]:
numhorasconst = 4

# Redes neuronales Densas

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [ ]:
print("¿GPU detectada?:", tf.config.list_physical_devices('GPU'))

In [ ]:
print("Versión de TensorFlow:", tf.__version__)

In [ ]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps])     # Añado a X las listaa de listas de precios 
        y.append(data[i+n_steps, 3])    # Añado a y el precio de cierre de la siguiente hora  
    return np.array(X), np.array(y)     # Los transformo en arrays de numpy

In [ ]:
def preparar_datos(df, numhoras):
    data = df[['open', 'high', 'low', 'close', 'value']].values # Selecciono y transformo las columnas del dataframe en un array
    # Genero una matriz de listas, cada entrada de la matriz contiene n listas del tipo [open, high, low, close, value] siendo n = numhoras
    X, y = create_sequences(data, numhoras) 
    X_aux = []
    for i in X: # Tansformo la matriz de listas en una lista de listas, juntando las n listas de cada entrada en una
        aux = []
        for r in range(0, numhoras): # Transformo las n listas de esa entrada en una
            for elem in i[r]:
                aux.append(elem)
        X_aux.append(aux)   # Añado la lista transformada a la lista global
    X_aux = np.array(X_aux) # Transformo la lista en un array de numpy
    return X_aux, y

In [ ]:
def evalRedDensa(ytest, y_pred):
    y_pred = y_pred.flatten()   # Transforma una lista de listas en una lista de valores
    suma = 0
    n = len(y_pred)     # Obtengo el tamaño de y
    for i in range(0,n):    # Suma el error relativo de todas las predicciones
        suma = abs(y_pred[i] - ytest[i])/ytest[i] +  suma
    error_medio = suma/n    # Divido la suma entre el número de predicciones para calcular la media
    emp = error_medio*100   # Multiplico por 100 para obtener el error porcentual medio
    return emp

In [ ]:
strategy = tf.distribute.MirroredStrategy() # Replica tu modelo en todas las GPUs disponibles
print(f"Número de GPUs detectadas: {strategy.num_replicas_in_sync}")

In [ ]:
def opti_redes_densas_multi_gpu(epoch_array, batch_array, numhoras, X_train, y_train, X_vali, y_vali, X_test, y_test):
    best = float('inf') # Asigno a best un valor infinito de tipo float
    epoch_best = 0
    bacth_best = 0
    best_model = None
    training_results = []

    for e in epoch_array:   # Genero modelos con todos los epoch pasados como parámetro en el array
        for b in batch_array:   # Genero modelos con todos los batch_size pasados como parámetro en el array
            best_value_of_the15 = float('inf')  # Asigno a best_value_of_the25 un valor infinito de tipo float
            best_model_of_the15 = None
            with tf.device('/CPU:0'):
                for m in range(15): # Genero 15 modelos con cada de par epoch, bacth_size 
                    with strategy.scope():  # El código debajo de esta línea se ejecuta usando la estrategia que hay en el scope
                        model = Sequential()    # Declaro el modelo como secuencial
                        # Creo la capa de entrada con 128 neuronas, definiendo la forma de la entrada y y cuya función de activacion es Rectified Linear Unit
                        model.add(Dense(128, activation='relu', input_shape=(numhoras * 5,)))   
                        # Creo la capa interna con 64 neuronas y cuya función de activacion es Rectified Linear Unit, que selecciona el másximo entre 0 y el valor de la neurona
                        model.add(Dense(64, activation='relu'))
                        # Creo la capa de salida 
                        model.add(Dense(1))
                        # Compilo la red neuronal usando adam como optimizador y mape, Mean Absolute Percentage Error, como función que debe minimizar
                        model.compile(optimizer='adam', loss='mape')

                        history = model.fit(X_train, y_train, epochs=e, batch_size=b, validation_data=(X_vali, y_vali), shuffle=False)  # Entreno el modelo
                    y_pred = model.predict(X_test)  # Realizo una predicción usando los datos de test
                    valor = evalRedDensa(y_test, y_pred) # Evalúo el rendimiento del modelo

                    if valor < best_value_of_the15: # Si el rendimiento que obtengo es el mejor de esta iteración hasta ahora lo sustituyo y guardo el modelo
                        best_value_of_the15 = valor
                        best_model_of_the15 = model

                print(f"Epoch: {e}, Batch size: {b}, Value: {best_value_of_the15}") # Imprimo por pantalla la epoch, el batch_size y el mejor rendimiento obtenido para estos
                
                training_results.append({"epoch": e, # Guardo la información de la epoch, el batch_size, el numero de horas y el mejor rendimiento obtenido para estos
                                        "batch_size": b, 
                                        "hours": numhoras, 
                                        "value": best_value_of_the15})  
                
            with open('pasosdados.txt', 'w') as archivo:   
                archivo.write("epoch: "+str(e)+", batch_size:" + str(b))    # Escribo en un archivo de texto la epoch y el batch_size para los que acabo de entrenar el modelo
            if best_value_of_the15 < best:   # Si el rendimiento que he obtenido en la iteración es el mejor hasta ahora lo sustituyo 
                best = best_value_of_the15
                epoch_best = e  # Guardo su epoch
                bacth_best = b  # Guardo su batch_size
                best_model = best_model_of_the15    # Guardo el modelo en la variable
                if best < 0.75: # Si el rendimiento es mejor que 0,75, guardo el modelo
                    cadena_guardado = f"ModelosDensosOptiMultiGPUIMC/mi_modelo_densoIMC_Opti_e{e}_b{b}_v{round(best, 3)}_nh{numhoras}"
                    best_model.save(cadena_guardado + ".keras") # Guardo el modelo

    results_df = pd.DataFrame(training_results) # Transformo los datos guardados en un dataframe
    cadena = "desnsasH" + str(numhoras) + ".csv"
    results_df.to_csv(cadena, index=False)  # Guardo el dataframe con formato csv
    print("Resultados guardados en 'densas.csv'")   # Imprimo por pantalla un mensaje que indica que el dataframe ha sido guardado
    return epoch_best, bacth_best, best, best_model

In [ ]:
def opti_rd_h(h_array, epoch_array, batch_array):
    best = float('inf') # Asigno a best un valor infinito de tipo float
    epoch_best = 0
    bacth_best = 0
    h_best = 0
    best_model = None
    for i in h_array:   # Entreno modelos con el numero de horas en el array de horas
        Xtrain, ytrain = preparar_datos(df_train, i)    # Preparo los datos de entrenamiento
        Xvali, yvali = preparar_datos(df_vali, i)   # Preparo los datos de validación
        Xtest, ytest = preparar_datos(df_test, i)   # Preparo los datos de test
        # Entreno los modelos usando los datos preparados con la cantidad de horas correcta
        valores = opti_redes_densas_multi_gpu(epoch_array, batch_array, i, Xtrain, ytrain, Xvali, yvali, Xtest, ytest)
        if valores[2] < best:   # Si el rendimiento que he obtenido en la iteración es el mejor hasta ahora lo sustituyo 
            best = valores[2]
            epoch_best = valores[0] # Guardo su epoch
            bacth_best = valores[1] # Guardo su batch_size
            h_best = i  # Guardo el número de horas
            best_model = valores[3] # Guardo el modelo en la variable
            cadena_guardado = "ModelosDensosOptiMoreDataIMCBest/mi_modelo_densoIMC_Opti_e"+str(epoch_best)+"_b"+str(bacth_best)+"_h"+str(i)+"_v"+str(round(best, 3))+"_nh"+str(i)
            best_model.save(cadena_guardado+".keras")   # Guardo el modelo
        with open('pasosdadoshoras.txt', 'w') as archivo:   
            archivo.write("horas: "+str(i)+"\n")    # Escribo en un archivo de texto la hora para la que acabo de entrenar los modelos
    return best, epoch_best, bacth_best, h_best, best_model # Devuelvo el mejor rendimiento, epoch, batch_size, hora y modelo

In [ ]:
# Llamo a la función de entrenamiento principal, la primera es la lista con las horas, al segunda es la lista con las epoch y la tercera es la lista de los batch_size
data = opti_rd_h([1, 3, 5, 7, 10, 12, 14, 18, 21], [4, 6, 10, 14, 20, 40], [4, 8, 12, 16, 32, 64, 128, 256])
print(data) # Imprimo lo que devuelve la función
print("Ha terminado")   # Imprimo un mensaje indicando que la ejecución ha concluido